In [2]:
import pandas as pd

context = pd.read_json(
    "/Users/naimsassine/Desktop/DSAI/Belgian-law-finetuning/Data/jsonl_files/all_text.jsonl",
    lines=True,
)

In [13]:
import re

# Path to your text file
file_path = "/Users/naimsassine/Desktop/DSAI/keys"

# Read the content of the file
with open(file_path, "r") as file:
    content = file.read()

# Regular expressions to match the key, base, and version
mykey = r'mykey\s*=\s*"([^"]+)"'
mykey = re.search(mykey, content).group(1)

myorg = r'myorg\s*=\s*"([^"]+)"'
myorg = re.search(myorg, content).group(1)

In [26]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "what is the capital of brussels"},
    ],
)

In [33]:
response.choices[0].message.content

'Brussels is the capital of Belgium.'

In [34]:
from openai import OpenAI, OpenAIError

client = OpenAI(organization=myorg, api_key=mykey)


def generate_text(question, model="gpt-3.5-turbo"):
    prompt = """Tu es un assistant avec des connaissances approfondies dans le milieu légal Belge. 
    Je veux que tu me donnes 5 questions et leur réponses respectives en te basant sur le 
    text juridique suivant. Note bien que le texte peut être en Anglais, Francais
    ou Neerlandais, mais je veux que ta réponse soit en Francais. Il est aussi important que le format retourner soit le suivant : 
    question 1 : ...//
    reponse 1 : ...//
    question 2 : ...//
    reponse 2 : ...//
    ...
    Voila le texte juridique : """
    full_prompt = f"{prompt}\n\n{question}"

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": full_prompt},
            ],
        )
        return response.choices[0].message.content
    except OpenAIError as e:
        return f"An error occurred: {e}"


def generate_answers_and_write_to_file(df, question_column, output_file):
    with open(output_file, "w") as file:
        for index, row in df.iterrows():
            question = row[question_column]
            answer = generate_text(question)
            file.write(f"Question: {question}\nAnswer: {answer}\n\n")

In [37]:
generate_answers_and_write_to_file(context, "text", "full_text.txt")

In [52]:
df = pd.read_csv("questions_answers_adjusted.csv")

In [56]:
df.iloc[902].values

array(['Quelle est la limite de compétence de la Cour de Cassation ?',
       "Les audiences judiciaires peuvent être tenues à huis clos en cas d'infractions politiques ou de presse, sur la base d'un vote à l'unanimité."],
      dtype=object)

In [58]:
def remove_empty_lines(file_path):
    # Read the file and filter out empty lines
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    non_empty_lines = [line for line in lines if line.strip()]

    # Write the non-empty lines back to the file
    with open(file_path, "w", encoding="utf-8") as file:
        file.writelines(non_empty_lines)


# Replace 'full_text.txt' with the path to your text file
remove_empty_lines("full_text.txt")

In [71]:
def identify_discrepancies(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    question_count = 0
    response_count = 0
    discrepancies = []

    for i, line in enumerate(lines):
        if line.lower().startswith("question"):
            question_count += 1
        elif line.lower().startswith("reponse"):
            response_count += 1
        else:
            # If the line does not start with "question" or "reponse", it could be an error
            discrepancies.append(
                f"Line {i + 1}: {line.strip()} does not start with 'question' or 'reponse'."
            )
        if i % 2 != 0:
            if question_count != response_count:
                print(line)

    # Check for count discrepancies
    if question_count != response_count:
        discrepancies.append(
            f"Number of questions ({question_count}) does not match number of responses ({response_count})."
        )

    # Print the results
    if discrepancies:
        for discrepancy in discrepancies:
            print(discrepancy)
    else:
        print(
            "No discrepancies found. The number of questions and responses are equal."
        )


identify_discrepancies("full_text.txt")

No discrepancies found. The number of questions and responses are equal.


In [76]:
import csv
import re


def extract_to_csv(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as file:
        lines = file.readlines()

    data = []
    current_question = None
    current_answer = None

    for line in lines:
        line = line.strip()
        if line.lower().startswith("question"):
            if current_question and current_answer:
                data.append([current_question, current_answer])
            current_question = re.sub(
                r"^question \d+ : ", "", line, flags=re.IGNORECASE
            )
            current_answer = None
        elif line.lower().startswith("reponse"):
            current_answer = re.sub(r"^reponse \d+ : ", "", line, flags=re.IGNORECASE)

    # Add the last question and answer pair if they exist
    if current_question and current_answer:
        data.append([current_question, current_answer])

    # Write the data to a CSV file
    with open(output_file, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Question", "Answer"])
        writer.writerows(data)


extract_to_csv("full_text.txt", "output.csv")

In [78]:
second_thing = pd.read_csv("output.csv")